In [2]:
import pandas as pd
import sqlite3

In [3]:
connection = sqlite3.connect('../data/checking-logs.sqlite')

In [5]:
df = pd.read_sql(sql="""
                 SELECT c.uid, c.labname, MIN(c.timestamp) AS first_commit_ts, MIN(p.datetime) AS first_view_ts 
                 FROM pageviews AS p
                 RIGHT JOIN checker AS c ON c.uid = p.uid
                 WHERE c.status="ready" AND c.numTrials=1 AND 
                 c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
                 AND c.uid LIKE 'user_%'
                 GROUP BY c.uid, c.labname""", 
                 con=connection, parse_dates=['first_commit_ts', 'first_view_ts'])

In [7]:
df.to_sql(name="datamart", con=connection, if_exists="replace", index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [ ]:
test = df[df['first_view_ts'].notna()]  # or .notnull()
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [9]:
control = df[df['first_view_ts'].isna()]
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [ ]:
#control['first_view_ts']=test['first_view_ts'].mean()
control.first_view_ts = control["first_view_ts"].fillna(test['first_view_ts'].mean())
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


C:\Users\sofia\AppData\Local\Temp\ipykernel_19516\2609164563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control.first_view_ts = control["first_view_ts"].fillna(test['first_view_ts'].mean())


None

In [13]:
test.to_sql(name="test", con=connection, if_exists="replace", index=False)
control.to_sql(name="control", con=connection, if_exists="replace", index=False)

81

In [15]:
connection.close()